Installation de tensorflow

dossier pour les datas :

/data/cars_tests

/data/cars_train

Importation des librairies

In [84]:
# import libraries
from IPython.display import Image, display
import numpy as np
import os, sys, tarfile
from os.path import join
from PIL import ImageFile
import pandas as pd
from matplotlib import cm
import seaborn as sns
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50, vgg16
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_auc_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
from sklearn import svm
from sklearn.mixture import GaussianMixture
from sklearn.isotonic import IsotonicRegression
import re
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, date, time
import pandas as pd
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

Récupération du dataset

In [ ]:
!wget http://ai.stanford.edu/~jkrause/car196/car_ims.tgz /content/sample_data/car_ims.tgz

--2022-07-29 07:46:30--  http://ai.stanford.edu/~jkrause/car196/car_ims.tgz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1956628579 (1.8G) [application/x-gzip]
Saving to: ‘car_ims.tgz’

car_ims.tgz         100%[===================>]   1.82G  21.8MB/s    in 87s     

2022-07-29 07:47:57 (21.5 MB/s) - ‘car_ims.tgz’ saved [1956628579/1956628579]

/content/sample_data/car_ims.tgz: Scheme missing.
FINISHED --2022-07-29 07:47:57--
Total wall clock time: 1m 27s
Downloaded: 1 files, 1.8G in 1m 27s (21.5 MB/s)


Extraction du dossier

In [ ]:
def extract(tar_url, extract_path='/content/sample_data/data'):
    tar = tarfile.open(tar_url, 'r')
    for item in tar:
        tar.extract(item, extract_path)
        if item.name.find(".tgz") != -1 or item.name.find(".tar") != -1:
            extract(item.name, "./" + item.name[:item.name.rfind('/')])
try:

    extract('/content/sample_data/car_ims.tgz')
    print ('Done.')
except:
    name = os.path.basename(sys.argv[0])
    print (name[:name.rfind('.')], '<filename>')

ipykernel_launcher <filename>


Récupération des data d'entrainement

In [ ]:
# recup all data for the training set
local_folder_train = '/content/sample_data/data/car_ims/'
train_img_paths_n = [join(local_folder_train,filename) for filename in os.listdir(local_folder_train)]
print(len(train_img_paths_n))
# pre_trained_model = InceptionV3(
#     input_shape=(150, 150, 3), include_top=False, weights=None)
# pre_trained_model.load_weights(local_weights_file)


16185


In [ ]:
#import mat table
mat = scipy.io.loadmat('/content/cars_annos.mat')
annot = mat['annotations']
annotd = annot.dtype
classn = mat['class_names']
classnd = classn.dtype
annotation = {n: annot[n][0, 0] for n in annotd.names}
print(len(classn[0]))


196


In [ ]:
# transform mat table to dict array
j = 0
for k, v in annotation.items() :
    value = []
    for a in annot :
        for i in range(0,len(a)) :
            if j >= 1 :
                value.append(a[i][j][0][0])
            else :
                value.append(a[i][j][0])
    annotation[k] = value
    j += 1
annotation

{'bbox_x1': [112,
  48,
  7,
  33,
  5,
  10,
  56,
  1,
  15,
  14,
  5,
  29,
  1,
  9,
  4,
  12,
  1,
  46,
  86,
  26,
  50,
  6,
  52,
  3,
  60,
  21,
  11,
  8,
  30,
  5,
  125,
  20,
  51,
  79,
  18,
  13,
  6,
  26,
  61,
  111,
  178,
  9,
  28,
  4,
  6,
  14,
  10,
  111,
  9,
  9,
  15,
  11,
  92,
  5,
  8,
  70,
  87,
  28,
  11,
  53,
  31,
  4,
  8,
  11,
  28,
  45,
  3,
  18,
  24,
  31,
  17,
  6,
  111,
  32,
  2,
  26,
  46,
  25,
  20,
  5,
  62,
  5,
  67,
  40,
  12,
  26,
  24,
  2,
  11,
  434,
  104,
  7,
  8,
  150,
  139,
  2,
  115,
  84,
  13,
  30,
  32,
  16,
  34,
  179,
  68,
  139,
  60,
  8,
  36,
  130,
  23,
  45,
  10,
  18,
  56,
  73,
  97,
  45,
  20,
  38,
  18,
  108,
  85,
  208,
  24,
  27,
  97,
  24,
  30,
  51,
  186,
  56,
  12,
  2,
  32,
  29,
  14,
  73,
  42,
  41,
  76,
  51,
  129,
  13,
  13,
  28,
  214,
  115,
  129,
  11,
  37,
  85,
  30,
  29,
  13,
  10,
  10,
  36,
  57,
  147,
  18,
  47,
  131,
  33,
  108,
  48,
  

In [ ]:
# df_annot
columns = [n for n in annotation.keys()]
df_annot = pd.DataFrame(annotation, columns=columns)
df_annot

,relative_im_path,bbox_x1,bbox_y1,bbox_x2,bbox_y2,class,test
0,car_ims/000001.jpg,112,7,853,717,1,0
1,car_ims/000002.jpg,48,24,441,202,1,0
2,car_ims/000003.jpg,7,4,277,180,1,0
3,car_ims/000004.jpg,33,50,197,150,1,0
4,car_ims/000005.jpg,5,8,83,58,1,0
...,...,...,...,...,...,...,...
16180,car_ims/016181.jpg,38,36,375,234,-60,1
16181,car_ims/016182.jpg,29,34,235,164,-60,1
16182,car_ims/016183.jpg,25,32,587,359,-60,1
16183,car_ims/016184.jpg,56,60,208,186,-60,1


In [ ]:
#df for class_names
class_names = []
for i in range(0,len(classn[0])) :
    class_names.append(classn[0][i][0])
c = np.array(class_names)
columns = ['class']
df_class = pd.DataFrame()
df_class['class'] = c.tolist()
df_class

,class
0,AM General Hummer SUV 2000
1,Acura RL Sedan 2012
2,Acura TL Sedan 2012
3,Acura TL Type-S 2008
4,Acura TSX Sedan 2012
...,...
191,Volkswagen Beetle Hatchback 2012
192,Volvo C30 Hatchback 2012
193,Volvo 240 Sedan 1993
194,Volvo XC90 SUV 2007


In [ ]:
# df en deux suivant column test
df_train = pd.DataFrame()
#print(df_annot)
df_train['img'] = df_annot.relative_im_path
df_train['class'] = df_annot['class']
selRows = df_train[df_train['class'] <= -60].index
df_train = df_train.drop(selRows, axis=0)
for i in range(0,196) :
    a= i+1
    df_train['class'] = df_train['class'].replace({a:df_class['class'][i]})
df_train

,img,class
0,car_ims/000001.jpg,AM General Hummer SUV 2000
1,car_ims/000002.jpg,AM General Hummer SUV 2000
2,car_ims/000003.jpg,AM General Hummer SUV 2000
3,car_ims/000004.jpg,AM General Hummer SUV 2000
4,car_ims/000005.jpg,AM General Hummer SUV 2000
...,...,...
10512,car_ims/010513.jpg,Honda Odyssey Minivan 2007
10513,car_ims/010514.jpg,Honda Odyssey Minivan 2007
10514,car_ims/010515.jpg,Honda Odyssey Minivan 2007
10515,car_ims/010516.jpg,Honda Odyssey Minivan 2007


In [ ]:
df_test = pd.DataFrame()
df_test['img'] = df_annot.relative_im_path[df_annot.test == 1]
df_test['class'] = df_annot['class'][df_annot.test == 1]
for i in range(0,196) :
    a= i+1
    df_test['class'] = df_test['class'].replace({a:df_class['class'][i]})
df_test

,img,class
45,car_ims/000046.jpg,AM General Hummer SUV 2000
46,car_ims/000047.jpg,AM General Hummer SUV 2000
47,car_ims/000048.jpg,AM General Hummer SUV 2000
48,car_ims/000049.jpg,AM General Hummer SUV 2000
49,car_ims/000050.jpg,AM General Hummer SUV 2000
...,...,...
16180,car_ims/016181.jpg,-60
16181,car_ims/016182.jpg,-60
16182,car_ims/016183.jpg,-60
16183,car_ims/016184.jpg,-60


In [ ]:
# image data generator : .1/255
from keras_preprocessing.image import ImageDataGenerator

datagen=ImageDataGenerator(rescale=.1/255, validation_split=0.2)
datagen


In [ ]:
train_generator=datagen.flow_from_dataframe(
    dataframe=df_train,
    directory="/content/sample_data/data/",
    x_col="img",
    y_col="class",
    subset="training",
    target_size=(150, 150),
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical")
    
valid_generator=datagen.flow_from_dataframe(
    dataframe=df_train,
    directory="/content/sample_data/data/",
    x_col="img",
    y_col="class",
    subset="validation",
    target_size=(150, 150),
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical")

# test_datagen=ImageDataGenerator(rescale=1./255.)
# test_generator=test_datagen.flow_from_dataframe(
#     dataframe=df_test,
#     directory="./data/",
#     x_col="img",
#     y_col=None,
#     batch_size=32,
#     seed=42,
#     shuffle=False,
#     class_mode=None)

Found 8414 validated image filenames belonging to 127 classes.
Found 2103 validated image filenames belonging to 127 classes.


In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model

In [ ]:
# Our input feature map is 150x150x3: 150x150 for the image pixels, and 3 for
# the three color channels: R, G, and B
img_input = layers.Input(shape=(150, 150, 3))

# First convolution extracts 16 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(16, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)

# Second convolution extracts 32 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

# Third convolution extracts 64 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

In [ ]:
# Flatten feature map to a 1-dim tensor so we can add fully connected layers
x = layers.Flatten()(x)

# Create a fully connected layer with ReLU activation and 512 hidden units
x = layers.Dense(512, activation='relu')(x)

# Create output layer with a single node and sigmoid activation
output = layers.Dense(1, activation='sigmoid')(x)

# Create model:
# input = input feature map
# output = input feature map + stacked convolution/maxpooling layers + fully 
# connected layer + sigmoid output layer
model = Model(img_input, output)


In [87]:
vgg = vgg16.VGG16(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax'
)
model2 = Model(img_input, vgg)

ValueError: ignored

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv2d_3 (Conv2D)           (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 74, 74, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 34, 34, 64)        1849

In [ ]:
from tensorflow.keras.optimizers import RMSprop

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [77]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

Epoch 1/10
262/262 [==============================] - 269s 1s/step - loss: 0.0513 - acc: 0.9904 - val_loss: 0.0474 - val_acc: 0.9921
Epoch 2/10
262/262 [==============================] - 259s 987ms/step - loss: 0.0469 - acc: 0.9921 - val_loss: 0.0469 - val_acc: 0.9921
Epoch 3/10
262/262 [==============================] - 257s 979ms/step - loss: 0.0466 - acc: 0.9921 - val_loss: 0.0466 - val_acc: 0.9921
Epoch 4/10
262/262 [==============================] - 256s 976ms/step - loss: 0.0464 - acc: 0.9921 - val_loss: 0.0464 - val_acc: 0.9921
Epoch 5/10
262/262 [==============================] - 260s 991ms/step - loss: 0.0463 - acc: 0.9921 - val_loss: 0.0463 - val_acc: 0.9921
Epoch 6/10
262/262 [==============================] - 256s 975ms/step - loss: 0.0462 - acc: 0.9921 - val_loss: 0.0462 - val_acc: 0.9921
Epoch 7/10
262/262 [==============================] - 255s 974ms/step - loss: 0.0462 - acc: 0.9921 - val_loss: 0.0462 - val_acc: 0.9921
Epoch 8/10
262/262 [==============================]

In [88]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

In [92]:
img_path = '/content/img.jpg'
img = image.load_img(img_path, target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = model.predict(x)

# print('Predicted:', decode_predictions(preds, top=3)[0])

[[[[-57.939003  -58.779     -63.68     ]
   [-64.939     -64.779     -69.68     ]
   [-18.939003  -22.779     -25.68     ]
   ...
   [145.061     131.22101   126.32     ]
   [145.061     131.22101   126.32     ]
   [145.061     131.22101   126.32     ]]

  [[-65.939     -58.779     -60.68     ]
   [-13.939003   -2.7789993  -3.6800003]
   [ -4.939003   20.221      17.32     ]
   ...
   [144.061     130.22101   125.32     ]
   [144.061     130.22101   125.32     ]
   [144.061     130.22101   125.32     ]]

  [[-23.939003  -36.779     -43.68     ]
   [ -0.939003   -6.7789993 -10.68     ]
   [-24.939003  -25.779     -30.68     ]
   ...
   [144.061     131.22101   124.32     ]
   [143.061     129.22101   124.32     ]
   [144.061     130.22101   125.32     ]]

  ...

  [[ 17.060997    9.221001    1.3199997]
   [ 26.060997   15.221001    8.32     ]
   [ 35.060997   22.221      15.32     ]
   ...
   [ 32.060997   18.221      13.32     ]
   [ 31.060997   17.221      12.32     ]
   [ 36.060997  

ValueError: ignored

In [126]:
from  tensorflow.keras.applications.resnet import ResNet101
train_generator=datagen.flow_from_dataframe(
    dataframe=df_train,
    directory="/content/sample_data/data/",
    x_col="img",
    y_col="class",
    subset="training",
    target_size=(224, 224),
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical")
    
valid_generator=datagen.flow_from_dataframe(
    dataframe=df_train,
    directory="/content/sample_data/data/",
    x_col="img",
    y_col="class",
    subset="validation",
    target_size=(224, 224),
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical")
model2 = ResNet101()
model2.summary()


# img_path = '/content/img.jpg'
# img = image.load_img(img_path, target_size=(224, 224))
# x = image.img_to_array(img)
# x = np.expand_dims(x, axis=0)
# x = preprocess_input(x)

# features = model2.predict(x)
# print('Predicted:', decode_predictions(features)[0])

Found 8414 validated image filenames belonging to 127 classes.
Found 2103 validated image filenames belonging to 127 classes.
179658752/179648224 [==============================] - 2s 0us/step
Model: "resnet101"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_16 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_16[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']           

In [120]:
datagen

In [127]:
cnn = Sequential()
# cnn.get_config()
for layer in model.layers[:-1]:
    cnn.add(layer)
# cnn.summary()

In [128]:
for layer in cnn.layers:
    layer.trainable = False
cnn.add(Dense(units=127, activation='softmax'))


In [129]:
cnn.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
cnn.fit(train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)

Epoch 1/2
220/262 [========================>.....] - ETA: 11:50 - loss: 5.7385 - accuracy: 0.0099

In [ ]:
# Distribution graphs (histogram/bar graph) of column data
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    # For displaying purposes, pick columns that have between 1 and 50 unique values
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]]
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num=None, figsize=(6 * nGraphPerRow, 8 * nGraphRow),
            dpi=80, facecolor='w', edgecolor='k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation=90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)
    plt.show()

# Correlation matrix
def plotCorrelationMatrix(df, graphWidth):
    filename = df.dataframeName
    df = df.dropna('columns')  # drop columns with NaN
    # keep columns where there are more than 1 unique values
    df = df[[col for col in df if df[col].nunique() > 1]]
    if df.shape[1] < 2:
        print(
            f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth),
            dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum=1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()


# Scatter and density plots
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include=[np.number])  # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    # keep columns where there are more than 1 unique values
    df = df[[col for col in df if df[col].nunique() > 1]]
    columnNames = list(df)
    if len(columnNames) > 10:  # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[
                                    plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k=1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2),
                        xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()
